In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

In [5]:
# 14 seconds
food = pd.read_csv("../../data/branded_food.csv")
food.info()
food.head(1)

C:\Users\manik\AppData\Local\Temp\ipykernel_7636\2822180072.py:2: DtypeWarning: Columns (2,3,4,6,9,12,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  food = pd.read_csv("../../data/branded_food.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845297 entries, 0 to 1845296
Data columns (total 20 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   fdc_id                       int64  
 1   brand_owner                  object 
 2   brand_name                   object 
 3   subbrand_name                object 
 4   gtin_upc                     object 
 5   ingredients                  object 
 6   not_a_significant_source_of  object 
 7   serving_size                 float64
 8   serving_size_unit            object 
 9   household_serving_fulltext   object 
 10  branded_food_category        object 
 11  data_source                  object 
 12  package_weight               object 
 13  modified_date                object 
 14  available_date               object 
 15  market_country               object 
 16  discontinued_date            float64
 17  preparation_state_code       object 
 18  trade_channel                object 
 19  

,fdc_id,brand_owner,brand_name,subbrand_name,gtin_upc,ingredients,not_a_significant_source_of,serving_size,serving_size_unit,household_serving_fulltext,branded_food_category,data_source,package_weight,modified_date,available_date,market_country,discontinued_date,preparation_state_code,trade_channel,short_description
0,1105904,Richardson Oilseed Products (US) Limited,NaN,NaN,27000612323,Vegetable Oil,NaN,15.0,ml,NaN,Oils Edible,GDSN,NaN,2020-10-02,2020-11-13,United States,NaN,NaN,NaN,NaN


In [6]:
food = food[['fdc_id','ingredients','branded_food_category']]
food.info()
food.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1845297 entries, 0 to 1845296
Data columns (total 3 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   fdc_id                 int64 
 1   ingredients            object
 2   branded_food_category  object
dtypes: int64(1), object(2)
memory usage: 42.2+ MB


,fdc_id,ingredients,branded_food_category
0,1105904,Vegetable Oil,Oils Edible


In [7]:
related_to_fruits_and_vegetables = [
    'Frozen Fruit & Fruit Juice Concentrates',
    'Canned Fruit',
    'Frozen Vegetables',
    'Fruit & Vegetable Juice, Nectars & Fruit Drinks',
    'Pre-Packaged Fruit & Vegetables',
    'Vegetable and Lentil Mixes',
    'Jam, Jelly & Fruit Spreads',
    'Canned Vegetables',
    'Fruits, Vegetables & Produce',
    'Fruit/Nuts/Seeds Combination',
    'Fruits Unprepared/Unprocessed (Frozen)',
    'Vegetables - Prepared/Processed (Frozen)',
    'Vegetables - Prepared/Processed (Shelf Stable)',
    'Vegetables - Unprepared/Unprocessed (Frozen)',
    'Vegetables - Prepared/Processed',
    'Fruit - Prepared/Processed',
    'Vegetables - Unprepared/Unprocessed (Frozen)',
    'Vegetables - Unprepared/Unprocessed (Shelf Stable)',
    'Vegetables Unprepared/Unprocessed (Shelf Stable)',
    'Fruit',
    'Frozen Fruit',
    'Wrapped Snacks - Fruit Bars',
    'Fresh Fruit and Vegetables',
    'Snack Foods - Dried Fruit',
    'Berries/Small Fruit',
    'Fruits Unprepared/Unprocessed (Shelf Stable)',
    'Fruit/Nuts/Seeds Mixes - Prepared/Processed (Shelf Stable)',
    'Fruit - Prepared/Processed (Frozen)',
    'Fruit Juice - Ready to Drink (Shelf Stable)',
    'Fruit Juice - Not Ready to Drink (Frozen)',
    'Oils Edible - Vegetable or Plant (Shelf Stable)'
]


In [8]:
fruits_vegs = food[food.branded_food_category.apply(lambda x:True if x in related_to_fruits_and_vegetables else False)]
fruits_vegs = fruits_vegs.drop('branded_food_category',axis = 1).dropna()
fruits_vegs.info()
fruits_vegs.head()

<class 'pandas.core.frame.DataFrame'>
Index: 149058 entries, 176 to 1845271
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   fdc_id       149058 non-null  int64 
 1   ingredients  149058 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.4+ MB


,fdc_id,ingredients
176,1106080,"ORGANIC STRAWBERRIES, ORGANIC PEACHES, ORGANIC..."
191,1106095,"PINEAPPLE, WATER, SUGAR, AND CITRIC ACID."
192,1106096,"SUGAR SNAP PEAS, CARROTS, ONIONS, MUSHROOMS."
238,1106142,BLUEBERRIES.
257,1106161,"ORGANIC PEARS, ORGANIC PEACHES, ORGANIC PUMPKI..."


In [9]:
#40 seconds
food_nutrient = pd.read_csv("../../data/food_nutrient.csv")
food_nutrient = food_nutrient.iloc[:,:4]
food_nutrient.info()
food_nutrient.head(1)

C:\Users\manik\AppData\Local\Temp\ipykernel_7636\372001563.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient = pd.read_csv("../../data/food_nutrient.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24743726 entries, 0 to 24743725
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   fdc_id       int64  
 2   nutrient_id  int64  
 3   amount       float64
dtypes: float64(1), int64(3)
memory usage: 755.1 MB


,id,fdc_id,nutrient_id,amount
0,13706913,1105904,1003,0.0


In [10]:
nutrient = pd.read_csv("../../data/nutrient.csv")
nutrient = nutrient.iloc[:,:2]
nutrient.info()
nutrient.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      474 non-null    int64 
 1   name    474 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.5+ KB


,id,name
0,2047,Energy (Atwater General Factors)


In [11]:
nutrient_merged = food_nutrient.merge(nutrient,left_on = 'nutrient_id',right_on = 'id',how = 'inner')
nutrient_merged = nutrient_merged.drop(['id_x','id_y','nutrient_id'],axis = 'columns')
nutrient_merged.info()
nutrient_merged.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24743714 entries, 0 to 24743713
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   fdc_id  int64  
 1   amount  float64
 2   name    object 
dtypes: float64(1), int64(1), object(1)
memory usage: 566.3+ MB


,fdc_id,amount,name
0,1105904,0.0,Protein


In [12]:
fruits_vegs

,fdc_id,ingredients
176,1106080,"ORGANIC STRAWBERRIES, ORGANIC PEACHES, ORGANIC..."
191,1106095,"PINEAPPLE, WATER, SUGAR, AND CITRIC ACID."
192,1106096,"SUGAR SNAP PEAS, CARROTS, ONIONS, MUSHROOMS."
238,1106142,BLUEBERRIES.
257,1106161,"ORGANIC PEARS, ORGANIC PEACHES, ORGANIC PUMPKI..."
...,...,...
1845201,2554819,"GREEN CABBAGE, FILTERED WATER SEA SALT, CARAWA..."
1845205,2554823,"PURIFIED WATER, ERYTHRITOL, PHYTOCOLLAGEN, STR..."
1845253,2554871,ORGANIC GREEN PEAS
1845260,2554878,"CANTALOUPE, HONEYDEW, PINEAPPLE, MANGO, STRAWB..."


In [13]:
nutrient_merged

,fdc_id,amount,name
0,1105904,0.00,Protein
1,1105904,93.33,Total lipid (fat)
2,1105904,0.00,"Carbohydrate, by difference"
3,1105904,0.00,"Vitamin D (D2 + D3), International Units"
4,1105904,0.00,"Fiber, total dietary"
...,...,...,...
24743709,2554914,0.00,"Vitamin D (D2 + D3), International Units"
24743710,2554914,13.33,"Carbohydrate, by difference"
24743711,2554914,0.00,Cholesterol
24743712,2554914,23.33,Protein


In [14]:
food_nutrient_merged = fruits_vegs.merge(nutrient_merged,on = 'fdc_id',how = 'inner')
food_nutrient_merged.info()
food_nutrient_merged.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826803 entries, 0 to 1826802
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   fdc_id       int64  
 1   ingredients  object 
 2   amount       float64
 3   name         object 
dtypes: float64(1), int64(1), object(2)
memory usage: 55.7+ MB


,fdc_id,ingredients,amount,name
0,1106080,"ORGANIC STRAWBERRIES, ORGANIC PEACHES, ORGANIC...",64.0,Energy
1,1106080,"ORGANIC STRAWBERRIES, ORGANIC PEACHES, ORGANIC...",0.0,Total lipid (fat)


In [15]:
food_nutrient_merged.set_index(['fdc_id','ingredients']).duplicated().sum()

1812911

In [16]:
food_nutrient_merged.fdc_id.duplicated().sum()

1685740

In [17]:
piv = food_nutrient_merged[['fdc_id','name','amount']].pivot(columns='name',values='amount')
piv.head(2)

name,Biotin,Caffeine,"Calcium, Ca","Carbohydrate, by difference","Carbohydrate, other","Chlorine, Cl",Cholesterol,"Choline, total","Chromium, Cr","Copper, Cu",...,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),"Vitamin D (D2 + D3), International Units",Vitamin D3 (cholecalciferol),Vitamin E,Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Water,Xylitol,"Zinc, Zn"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
final = pd.concat([food_nutrient_merged[['fdc_id','ingredients']], piv], axis=1)
final = final.groupby(['fdc_id','ingredients']).mean().reset_index()
final = final.set_index('fdc_id')
final = final[~final.Energy.isna()]
final

,ingredients,Biotin,Caffeine,"Calcium, Ca","Carbohydrate, by difference","Carbohydrate, other","Chlorine, Cl",Cholesterol,"Choline, total","Chromium, Cr",...,"Vitamin C, total ascorbic acid",Vitamin D (D2 + D3),"Vitamin D (D2 + D3), International Units",Vitamin D3 (cholecalciferol),Vitamin E,Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),Water,Xylitol,"Zinc, Zn"
fdc_id,,,,,,,,,,,,,,,,,,,,,
346432,"Prepared White Beans (Water, White Beans), Chi...",NaN,NaN,45.0,15.00,NaN,NaN,16.0,NaN,NaN,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349015,"Organic Tomatoes, Organic Tomato Puree, Organi...",NaN,NaN,33.0,8.33,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349570,"Potatoes*, Onions, Dextrose, Calcium Stearoyl...",NaN,NaN,NaN,83.33,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356531,"PEARS, WATER RED WINE, SUGAR, SPICES AND CITRI...",NaN,NaN,14.0,21.43,NaN,NaN,0.0,NaN,NaN,...,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356532,"BLACKBERRY, COFFEE, SUGAR, WATER AND CITRIC AC...",NaN,NaN,0.0,53.33,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554819,"GREEN CABBAGE, FILTERED WATER SEA SALT, CARAWA...",NaN,NaN,33.0,1.67,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2554823,"PURIFIED WATER, ERYTHRITOL, PHYTOCOLLAGEN, STR...",NaN,NaN,16.0,10.67,NaN,NaN,0.0,NaN,NaN,...,240.0,NaN,0.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2554871,ORGANIC GREEN PEAS,NaN,NaN,24.0,14.12,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
for i in final.columns:
    if final[i].isna().sum() > 50000:
        final.drop(i,axis = 'columns',inplace= True)

In [32]:
final.reset_index()['ingredients'].duplicated().sum()

119755

In [33]:
final = final.fillna(0)

In [34]:
final.head(1)

,ingredients,"Calcium, Ca","Carbohydrate, by difference",Cholesterol,Energy,"Fatty acids, total saturated","Fatty acids, total trans","Fiber, total dietary","Iron, Fe",Protein,"Sodium, Na","Sugars, total including NLEA",Total lipid (fat)
fdc_id,,,,,,,,,,,,,
346432,"Prepared White Beans (Water, White Beans), Chi...",45.0,15.0,16.0,118.0,1.14,0.0,4.1,0.82,0.0,386.0,3.64,3.64


In [35]:
final = final.rename(columns={'Calcium, Ca':'Calcium',
                      'Carbohydrate, by difference':"Carbohydrate",
                      "Fatty acids, total saturated":"Fatty_acids_total_saturated",
                      "Fatty acids, total trans":"Fatty_acids_total_trans",
                      "Fiber, total dietary":"Fiber",
                      'Iron, Fe':'Iron',
                      'Sodium, Na':'Sodium',
                      'Sugars, total including NLEA':'Sugars',
                      "Total lipid (fat)":'Fat'
                      })

In [55]:
# final.to_csv('PredictionTable.csv')

In [36]:
X = final.drop(['ingredients', 'Energy'],axis = 1)
Y = final.Energy.to_frame()

In [37]:
X

,Calcium,Carbohydrate,Cholesterol,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat
fdc_id,,,,,,,,,,,
346432,45.0,15.00,16.0,1.14,0.0,4.1,0.82,0.00,386.0,3.64,3.64
349015,33.0,8.33,0.0,0.00,0.0,1.7,1.80,1.67,183.0,3.33,0.00
349570,0.0,83.33,0.0,0.00,0.0,4.2,0.00,8.33,62.0,4.17,0.00
356531,14.0,21.43,0.0,0.00,0.0,0.7,0.26,0.71,7.0,19.29,0.00
356532,0.0,53.33,0.0,0.00,0.0,0.0,0.00,0.00,0.0,46.67,0.00
...,...,...,...,...,...,...,...,...,...,...,...
2554819,33.0,1.67,0.0,0.00,0.0,1.7,0.00,1.67,783.0,0.00,0.00
2554823,16.0,10.67,0.0,0.00,0.0,0.7,0.13,0.00,59.0,0.00,0.00
2554871,24.0,14.12,0.0,0.00,0.0,4.7,1.18,4.71,71.0,4.71,0.00


<h1>Model Building

In [38]:
acc = list()
for i in range(50):
    x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=i)
    lr = LinearRegression()
    lr.fit(x_train,y_train)
    pred = lr.predict(x_test)
    acc.append([i,r2_score(y_test,pred)])

In [39]:
acc.sort(key=lambda x:x[1],reverse=True)
acc[:5]

[[30, 0.9931971979131886],
 [8, 0.9930260320200711],
 [19, 0.9924087689785367],
 [29, 0.9918236889797555],
 [45, 0.9917209339189793]]

In [40]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=30)
lr = LinearRegression()
lr.fit(x_train,y_train)
pred = lr.predict(x_test)
score = r2_score(y_test,pred)
print(score)

0.9931971979131886


In [41]:
# from sklearn.ensemble import RandomForestRegressor
# x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=30)
# RFR = RandomForestRegressor(max_depth=5)
# RFR.fit(x_train,y_train)
# pred = RFR.predict(x_test)
# r2_score(y_test,pred)

<h1>Recommendation System Building 

In [42]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
final.head(1)

,ingredients,Calcium,Carbohydrate,Cholesterol,Energy,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat
fdc_id,,,,,,,,,,,,,
346432,"Prepared White Beans (Water, White Beans), Chi...",45.0,15.0,16.0,118.0,1.14,0.0,4.1,0.82,0.0,386.0,3.64,3.64


In [44]:
import re

final_copy = final.copy()

def clean_string(input_string):
    # Convert to lowercase
    input_string = input_string.lower()

    # Remove leading numbers, commas, and colons
    cleaned_string = re.sub(r'^[0-9]+[:,\s]*', '', input_string)

    # Remove content within parentheses
    cleaned_string = re.sub(r'\([^)]*\)', '', cleaned_string)
    
    # Remove content within square brackets
    cleaned_string = re.sub(r'\[[^\]]*\]', '', cleaned_string)

    # Remove unnecessary phrases, including "of:"
    cleaned_string = re.sub(r'\b(less than|more than|\d+% of:|ingredients:|of:)\b', '', cleaned_string, flags=re.IGNORECASE)

    # Remove asterisks and full stops
    cleaned_string = cleaned_string.replace('*', '').replace('.', '')

    # Remove "100%"
    cleaned_string = cleaned_string.replace('100%', '')

    # Remove leading and trailing spaces

    # Remove '%' symbols
    cleaned_string = cleaned_string.replace('%', '')

    # Remove additional specified strings, including "of:"
    cleaned_string = cleaned_string.replace('/ lb us # size b potatoes', '').replace('oz salt', '').replace('of:', '')

    # Replace multiple spaces with a single space
    cleaned_string = ' '.join(cleaned_string.split())

    # Remove the first character if it is not an alphabet character
    if cleaned_string and not cleaned_string[0].isalpha():
        cleaned_string = cleaned_string[1:]
    if cleaned_string and not cleaned_string[0].isalpha():
        cleaned_string = cleaned_string[1:]
    if cleaned_string and not cleaned_string[0].isalpha():
        cleaned_string = cleaned_string[1:]
    # Remove entire word if it contains "chicken" or "meat"
    cleaned_string = ' '.join(word for word in cleaned_string.split() if 'chicken' not in word and 'meat' not in word)

    cleaned_string = cleaned_string.strip()
    return cleaned_string

# Example usage for the provided list of strings
for i in range(len(final_copy)):
    li = final_copy.iloc[i, 0]
    cleaned_li = clean_string(li)
    final_copy.iloc[i, 0] = cleaned_li


In [45]:
temp = final_copy.groupby('ingredients').mean()
temp

,Calcium,Carbohydrate,Cholesterol,Energy,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat
ingredients,,,,,,,,,,,,
"a blend of grape, pineapple, pear and apple juice concentrates and other ingredients contains high fructose corn syrup, pure filtered water, concentrated grape juice, 1 concentrated pineapple, pear and apple juices, natural flavors, citric acid , grape skin extract , sugar",0.000000,50.00,0.000000,183.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,48.33,0.000000
"a mixture of arugula, broccoli, amaranth, beet tops, parsley, radish, kale or others",0.000000,0.00,0.000000,29.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000
"a seasonal blend of organic greens such as: organic kale, organic tuscan kale, organic red kale, organic collard, organic chard",146.000000,7.04,0.000000,42.000000,0.000000,0.0,3.5,1.410000,3.520,70.000000,1.41,0.700000
"a spicy yet flavorful sea salt blend ingredients: jalapeno chile, sea salt, garlic flakes, garlic granules, cilantro, organic rice hull, mexican oregano made from pure sea salt and a mix of 4 smoky sweet red chiles ingredients: new mexico chile, sea salt, pasilla negra chile, guajillo chile, ancho chile, organic rice hull",14.000000,17.57,0.000000,74.000000,0.000000,0.0,1.4,0.730000,2.030,0.000000,0.68,0.000000
a water extract of dried prunes,8.000000,20.83,0.000000,83.000000,0.000000,0.0,0.8,0.750000,0.420,15.000000,9.58,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
"zucchini, yellow squash, red onions, red bell peppers",7.500000,4.55,51.166667,36.166667,0.378333,0.0,1.1,0.205000,2.465,20.166667,2.27,0.946667
"zucchini, yellow squash, thyme",24.857143,7.50,0.000000,32.000000,0.000000,0.0,1.5,0.454286,1.180,6.000000,2.35,0.220000
"zucchini, yellow squash, yellow onion, dill, black pepper",21.000000,4.71,0.000000,24.000000,0.000000,0.0,1.2,0.000000,1.180,0.000000,2.35,0.000000


In [56]:
temp[['Calcium', 'Carbohydrate',
       'Cholesterol', 'Fatty_acids_total_saturated',
       'Fatty_acids_total_trans', 'Fiber', 'Iron',
       'Protein', 'Sodium', 'Sugars',
       'Fat', 'Energy']].reset_index().to_csv('RecommendationTable.csv')

In [61]:
dim_red_df = temp.copy()
dim_red_df = dim_red_df[['Calcium', 'Carbohydrate',
       'Cholesterol', 'Fatty_acids_total_saturated',
       'Fatty_acids_total_trans', 'Fiber', 'Iron',
       'Protein', 'Sodium', 'Sugars',
       'Fat', 'Energy']]
pca = PCA(3)
dim_red = pca.fit_transform(dim_red_df)
similarity = cosine_similarity(dim_red)
sorted(enumerate(similarity[0]), key = lambda x: x[1], reverse= True)

[(0, 1.0000000000000002),
 (8605, 0.9999999764171592),
 (6526, 0.9999992050304124),
 (6536, 0.9999992050304124),
 (18218, 0.9999992050304124),
 (18220, 0.9999992050304124),
 (13117, 0.9999981463274218),
 (9985, 0.9999975547540909),
 (3410, 0.9999971689373143),
 (592, 0.9999964002129254),
 (770, 0.9999964002129254),
 (9531, 0.9999964002129254),
 (12991, 0.9999960711090484),
 (14943, 0.9999957771202171),
 (2242, 0.9999942620028243),
 (11633, 0.9999942620028243),
 (13551, 0.9999942620028243),
 (15161, 0.9999942620028243),
 (15175, 0.9999942620028243),
 (18408, 0.9999934278427501),
 (3228, 0.9999925904457243),
 (3230, 0.9999925904457243),
 (15677, 0.9999925904457243),
 (2301, 0.9999883580377579),
 (9988, 0.9999883580377579),
 (3252, 0.9999800077826957),
 (287, 0.9999743585923787),
 (14909, 0.9999733796831828),
 (15084, 0.9999665019646362),
 (4407, 0.9999621010596796),
 (6309, 0.9999487817148517),
 (7369, 0.9999451446893219),
 (3789, 0.9999327515685518),
 (13937, 0.9999263589812356),
 (1357

In [62]:
temp.iloc[0]

Calcium                          0.00
Carbohydrate                    50.00
Cholesterol                      0.00
Energy                         183.00
Fatty_acids_total_saturated      0.00
Fatty_acids_total_trans          0.00
Fiber                            0.00
Iron                             0.00
Protein                          0.00
Sodium                           0.00
Sugars                          48.33
Fat                              0.00
Name: a blend of grape, pineapple, pear and apple juice concentrates and other ingredients contains high fructose corn syrup, pure filtered water, concentrated grape juice, 1 concentrated pineapple, pear and apple juices, natural flavors, citric acid , grape skin extract , sugar, dtype: float64

In [63]:
temp.iloc[6526]

Calcium                          0.00
Carbohydrate                    47.37
Cholesterol                      0.00
Energy                         184.00
Fatty_acids_total_saturated      0.00
Fatty_acids_total_trans          0.00
Fiber                            0.00
Iron                             0.00
Protein                          0.00
Sodium                           0.00
Sugars                          42.11
Fat                              0.00
Name: fruit syrup, blackberries, apple juice concentrate, fruit pectin, citric acid, natural flavor, calcium citrate, dtype: float64

In [64]:
cls = temp.reset_index()
cls

,ingredients,Calcium,Carbohydrate,Cholesterol,Energy,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat
0,"a blend of grape, pineapple, pear and apple ju...",0.000000,50.00,0.000000,183.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,48.33,0.000000
1,"a mixture of arugula, broccoli, amaranth, beet...",0.000000,0.00,0.000000,29.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000
2,a seasonal blend of organic greens such as: or...,146.000000,7.04,0.000000,42.000000,0.000000,0.0,3.5,1.410000,3.520,70.000000,1.41,0.700000
3,a spicy yet flavorful sea salt blend ingredien...,14.000000,17.57,0.000000,74.000000,0.000000,0.0,1.4,0.730000,2.030,0.000000,0.68,0.000000
4,a water extract of dried prunes,8.000000,20.83,0.000000,83.000000,0.000000,0.0,0.8,0.750000,0.420,15.000000,9.58,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18647,"zucchini, yellow squash, red onions, red bell ...",7.500000,4.55,51.166667,36.166667,0.378333,0.0,1.1,0.205000,2.465,20.166667,2.27,0.946667
18648,"zucchini, yellow squash, thyme",24.857143,7.50,0.000000,32.000000,0.000000,0.0,1.5,0.454286,1.180,6.000000,2.35,0.220000
18649,"zucchini, yellow squash, yellow onion, dill, b...",21.000000,4.71,0.000000,24.000000,0.000000,0.0,1.2,0.000000,1.180,0.000000,2.35,0.000000
18650,"zucchini, yellow summer squash",24.000000,3.53,0.000000,18.000000,0.000000,0.0,1.2,0.420000,1.180,0.000000,2.35,0.000000


In [65]:
LE = LabelEncoder()

In [66]:
cls.columns

Index(['ingredients', 'Calcium', 'Carbohydrate', 'Cholesterol', 'Energy',
       'Fatty_acids_total_saturated', 'Fatty_acids_total_trans', 'Fiber',
       'Iron', 'Protein', 'Sodium', 'Sugars', 'Fat'],
      dtype='object')

In [67]:
cls = cls[['Calcium', 'Carbohydrate',
       'Cholesterol', 'Fatty_acids_total_saturated',
       'Fatty_acids_total_trans', 'Fiber', 'Iron',
       'Protein', 'Sodium', 'Sugars',
       'Fat', 'Energy', 'ingredients']]

cls

,Calcium,Carbohydrate,Cholesterol,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat,Energy,ingredients
0,0.000000,50.00,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,48.33,0.000000,183.000000,"a blend of grape, pineapple, pear and apple ju..."
1,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,29.000000,"a mixture of arugula, broccoli, amaranth, beet..."
2,146.000000,7.04,0.000000,0.000000,0.0,3.5,1.410000,3.520,70.000000,1.41,0.700000,42.000000,a seasonal blend of organic greens such as: or...
3,14.000000,17.57,0.000000,0.000000,0.0,1.4,0.730000,2.030,0.000000,0.68,0.000000,74.000000,a spicy yet flavorful sea salt blend ingredien...
4,8.000000,20.83,0.000000,0.000000,0.0,0.8,0.750000,0.420,15.000000,9.58,0.000000,83.000000,a water extract of dried prunes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18647,7.500000,4.55,51.166667,0.378333,0.0,1.1,0.205000,2.465,20.166667,2.27,0.946667,36.166667,"zucchini, yellow squash, red onions, red bell ..."
18648,24.857143,7.50,0.000000,0.000000,0.0,1.5,0.454286,1.180,6.000000,2.35,0.220000,32.000000,"zucchini, yellow squash, thyme"
18649,21.000000,4.71,0.000000,0.000000,0.0,1.2,0.000000,1.180,0.000000,2.35,0.000000,24.000000,"zucchini, yellow squash, yellow onion, dill, b..."
18650,24.000000,3.53,0.000000,0.000000,0.0,1.2,0.420000,1.180,0.000000,2.35,0.000000,18.000000,"zucchini, yellow summer squash"


In [68]:
x_cls = cls.drop('ingredients',axis = 1)
y_cls = cls.ingredients

In [69]:
x_cls.shape

(18652, 12)

In [70]:
x_cls = pca.transform(x_cls)
x_cls.shape

(18652, 3)

In [71]:
y_cls = LE.fit_transform(y_cls)
y_cls

array([    0,     1,     2, ..., 18649, 18650, 18651])

In [72]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [73]:
#10 minutes to run
RF_clf = RandomForestClassifier(max_depth=15)
RF_clf.fit(x_cls, y_cls)

RandomForestClassifier(max_depth=15)

In [74]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()


In [75]:
# #10 minutes to run
# RF_clf = RandomForestClassifier(max_depth=20)
# RF_clf.fit(x_cls, y_cls)

In [76]:
# svc = SVC()
# svc.fit(x_cls, y_cls)

In [77]:
# fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (20,5), dpi=800)
# tree.plot_tree(RF_clf.estimators_[0],
#                filled = True)
# plt.show()

In [78]:
x_cls

array([[-112.71026161,   -8.41037091,  117.53277642],
       [-158.96870217,  -17.54581875,  -37.8920531 ],
       [ -67.47582663,  115.22681151,  -58.33618132],
       ...,
       [-157.10162362,    2.89920245,  -43.74897077],
       [-158.42220793,    5.50975155,  -49.8751628 ],
       [-125.37775981,    5.56940559,  -41.64583197]])

In [79]:
cls.drop('ingredients',axis = 1).iloc[[0]]

,Calcium,Carbohydrate,Cholesterol,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat,Energy
0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.33,0.0,183.0


In [80]:
target = cls[cls.ingredients ==LE.inverse_transform(RF_clf.predict(pca.transform(cls.drop('ingredients',axis = 1).iloc[[0]])))[0]]
target

,Calcium,Carbohydrate,Cholesterol,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat,Energy,ingredients
8605,0.0,44.74,0.0,0.0,0.0,0.7,0.0,0.0,0.0,44.74,0.0,184.0,"lingonberries, sugar, water, pectin"


In [81]:
target.ingredients.values[0]

'lingonberries, sugar, water, pectin'

In [82]:
target.index[0]

8605

In [83]:
cls[cls.ingredients == target.ingredients.values[0]].index[0]

8605

In [86]:
sorted(enumerate(similarity[8605]),key = lambda x: x[1], reverse = True)[:5]

[(8605, 1.0),
 (0, 0.9999999764171592),
 (6526, 0.999998907811474),
 (6536, 0.999998907811474),
 (18218, 0.999998907811474)]

In [91]:
cls.iloc[18218]

Calcium                                                                      0.0
Carbohydrate                                                               47.37
Cholesterol                                                                  0.0
Fatty_acids_total_saturated                                                  0.0
Fatty_acids_total_trans                                                      0.0
Fiber                                                                        0.0
Iron                                                                         0.0
Protein                                                                      0.0
Sodium                                                                       0.0
Sugars                                                                     42.11
Fat                                                                          0.0
Energy                                                                     184.0
ingredients                 

<h1>Saving Models

In [92]:
#Saving Linear Regresssion Prediction Model

with open('Prediction_Model.pkl','wb') as model_file:
    pickle.dump(lr,model_file)

In [93]:
#Saving RandomForest Classification Model

with open('Classification_Model.pkl','wb') as model_file:
    pickle.dump(RF_clf,model_file)

In [94]:
#saving similarity matrix

with open('similarity.pkl','wb') as model_file:
    pickle.dump(similarity,model_file)

In [95]:
#saving cls table
with open('classification_table.pkl','wb') as model_file:
    pickle.dump(cls,model_file)

In [96]:
#saving Label Encoder
with open('Labels.pkl','wb') as model_file:
    pickle.dump(LE,model_file)

In [97]:
#saving PCA
with open('pca.pkl','wb') as model_file:
    pickle.dump(pca,model_file)

In [98]:
cls

,Calcium,Carbohydrate,Cholesterol,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat,Energy,ingredients
0,0.000000,50.00,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,48.33,0.000000,183.000000,"a blend of grape, pineapple, pear and apple ju..."
1,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.00,0.000000,29.000000,"a mixture of arugula, broccoli, amaranth, beet..."
2,146.000000,7.04,0.000000,0.000000,0.0,3.5,1.410000,3.520,70.000000,1.41,0.700000,42.000000,a seasonal blend of organic greens such as: or...
3,14.000000,17.57,0.000000,0.000000,0.0,1.4,0.730000,2.030,0.000000,0.68,0.000000,74.000000,a spicy yet flavorful sea salt blend ingredien...
4,8.000000,20.83,0.000000,0.000000,0.0,0.8,0.750000,0.420,15.000000,9.58,0.000000,83.000000,a water extract of dried prunes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18647,7.500000,4.55,51.166667,0.378333,0.0,1.1,0.205000,2.465,20.166667,2.27,0.946667,36.166667,"zucchini, yellow squash, red onions, red bell ..."
18648,24.857143,7.50,0.000000,0.000000,0.0,1.5,0.454286,1.180,6.000000,2.35,0.220000,32.000000,"zucchini, yellow squash, thyme"
18649,21.000000,4.71,0.000000,0.000000,0.0,1.2,0.000000,1.180,0.000000,2.35,0.000000,24.000000,"zucchini, yellow squash, yellow onion, dill, b..."
18650,24.000000,3.53,0.000000,0.000000,0.0,1.2,0.420000,1.180,0.000000,2.35,0.000000,18.000000,"zucchini, yellow summer squash"


In [ ]:
final

,ingredients,Calcium,Carbohydrate,Cholesterol,Energy,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat
fdc_id,,,,,,,,,,,,,
346432,"Prepared White Beans (Water, White Beans), Chi...",45.0,15.00,16.0,118.0,1.14,0.0,4.1,0.82,0.00,386.0,3.64,3.64
349015,"Organic Tomatoes, Organic Tomato Puree, Organi...",33.0,8.33,0.0,42.0,0.00,0.0,1.7,1.80,1.67,183.0,3.33,0.00
349570,"Potatoes*, Onions, Dextrose, Calcium Stearoyl...",0.0,83.33,0.0,375.0,0.00,0.0,4.2,0.00,8.33,62.0,4.17,0.00
356531,"PEARS, WATER RED WINE, SUGAR, SPICES AND CITRI...",14.0,21.43,0.0,86.0,0.00,0.0,0.7,0.26,0.71,7.0,19.29,0.00
356532,"BLACKBERRY, COFFEE, SUGAR, WATER AND CITRIC AC...",0.0,53.33,0.0,200.0,0.00,0.0,0.0,0.00,0.00,0.0,46.67,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554819,"GREEN CABBAGE, FILTERED WATER SEA SALT, CARAWA...",33.0,1.67,0.0,17.0,0.00,0.0,1.7,0.00,1.67,783.0,0.00,0.00
2554823,"PURIFIED WATER, ERYTHRITOL, PHYTOCOLLAGEN, STR...",16.0,10.67,0.0,7.0,0.00,0.0,0.7,0.13,0.00,59.0,0.00,0.00
2554871,ORGANIC GREEN PEAS,24.0,14.12,0.0,82.0,0.00,0.0,4.7,1.18,4.71,71.0,4.71,0.00


In [ ]:
final

,ingredients,Calcium,Carbohydrate,Cholesterol,Energy,Fatty_acids_total_saturated,Fatty_acids_total_trans,Fiber,Iron,Protein,Sodium,Sugars,Fat
fdc_id,,,,,,,,,,,,,
346432,"Prepared White Beans (Water, White Beans), Chi...",45.0,15.00,16.0,118.0,1.14,0.0,4.1,0.82,0.00,386.0,3.64,3.64
349015,"Organic Tomatoes, Organic Tomato Puree, Organi...",33.0,8.33,0.0,42.0,0.00,0.0,1.7,1.80,1.67,183.0,3.33,0.00
349570,"Potatoes*, Onions, Dextrose, Calcium Stearoyl...",0.0,83.33,0.0,375.0,0.00,0.0,4.2,0.00,8.33,62.0,4.17,0.00
356531,"PEARS, WATER RED WINE, SUGAR, SPICES AND CITRI...",14.0,21.43,0.0,86.0,0.00,0.0,0.7,0.26,0.71,7.0,19.29,0.00
356532,"BLACKBERRY, COFFEE, SUGAR, WATER AND CITRIC AC...",0.0,53.33,0.0,200.0,0.00,0.0,0.0,0.00,0.00,0.0,46.67,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2554819,"GREEN CABBAGE, FILTERED WATER SEA SALT, CARAWA...",33.0,1.67,0.0,17.0,0.00,0.0,1.7,0.00,1.67,783.0,0.00,0.00
2554823,"PURIFIED WATER, ERYTHRITOL, PHYTOCOLLAGEN, STR...",16.0,10.67,0.0,7.0,0.00,0.0,0.7,0.13,0.00,59.0,0.00,0.00
2554871,ORGANIC GREEN PEAS,24.0,14.12,0.0,82.0,0.00,0.0,4.7,1.18,4.71,71.0,4.71,0.00


In [ ]:
rec = final[['ingredients','Energy']]
rec

,ingredients,Energy
fdc_id,,
346432,"Prepared White Beans (Water, White Beans), Chi...",118.0
349015,"Organic Tomatoes, Organic Tomato Puree, Organi...",42.0
349570,"Potatoes*, Onions, Dextrose, Calcium Stearoyl...",375.0
356531,"PEARS, WATER RED WINE, SUGAR, SPICES AND CITRI...",86.0
356532,"BLACKBERRY, COFFEE, SUGAR, WATER AND CITRIC AC...",200.0
...,...,...
2554819,"GREEN CABBAGE, FILTERED WATER SEA SALT, CARAWA...",17.0
2554823,"PURIFIED WATER, ERYTHRITOL, PHYTOCOLLAGEN, STR...",7.0
2554871,ORGANIC GREEN PEAS,82.0


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder 

In [ ]:
X = rec.drop('ingredients',axis = 1)
Y = rec.ingredients

In [ ]:
le = LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
Y

array([14904, 12767,     0, ..., 11666,  2913,   677])

In [ ]:
le.inverse_transform([Y[0]])

array(['Prepared White Beans (Water, White Beans), Chicken Franks (Mechanically Separated Chicken, Water, Salt, Corn Syrup, Dextrose, Sodium Phosphate, Natural Flavorings, Paprika, Sodium Erythorbate, Sodium Nitrite, Extractives of Paprika), Tomato Puree (Water, Tomato Paste), Water, Sugar, Less than 2% of: High Fructose Corn Syrup, Salt, Onion Powder, Natural and Artificial Flavors. MAY CONTAIN: SOY.'],
      dtype=object)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [ ]:
rf_clf = RandomForestClassifier(max_depth=15)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
rf_clf.fit(x_train,y_train)
pred = rf_clf.predict(x_test)
accuracy_score(y_test,pred)